In [1]:
from urllib.request import urlopen
import matplotlib.pyplot as plt
from collections import Counter
from bs4 import BeautifulSoup
import re
import pandas as pd

In [2]:
wiki="https://en.wikipedia.org/wiki/Gal_Gadot"

In [3]:
page = urlopen(wiki)

In [4]:
soup = BeautifulSoup(page)

In [5]:
right_table=soup.find('table', class_='wikitable sortable')

In [6]:
A=[]
B=[]
C=[]
D=[]
movies_links=[]
prev=2009
for row in right_table.findAll("tr"):
        directors=""
        cells=row.findAll('td')
        
        if len(cells)==5:
            A.append((cells[0].find(text=True)).strip('\n'))
            B.append((cells[1].find(text=True)).split('\n')[0])
            C.append((cells[2].find(text=True)).strip('\n'))
            directors=(cells[3].findAll(text=True))[0].strip('\n')
            if len(cells[3].findAll(text=True))>=3:
                directors=""
                for name in cells[3].findAll(text=True):  
                    directors+= name+" "
            D.append(directors.split('\n')[0])
            prev=cells[0].find(text=True)
            #for question 2
            movies_links.append(cells[1].find("a"))
#             print(cells[3].findAll(text=True))
            
        elif len(cells)==4:
            A.append((prev).strip('\n'))
            B.append((cells[0].find(text=True)).split('\n')[0])
            C.append((cells[1].find(text=True)).strip('\n'))
            directors=(cells[2].findAll(text=True))[0].strip('\n')
            if len(cells[2].findAll(text=True))>=3:
                directors=""
                for name in cells[2].findAll(text=True):  
                    directors+= name+" "  
            D.append(directors.split('\n')[0])
            #for question 2
            movies_links.append(cells[0].find("a"))
#             print(cells[2].findAll(text=True))

In [7]:

df=pd.DataFrame(A,columns=['Years'])
df['Title']=B
df['Role']=C
df['Director(s)']=D
df
#answer to question 1:

,Years,Title,Role,Director(s)
0,2009,Fast & Furious,Gisele Yashar,Justin Lin
1,2010,Date Night,Natanya,Shawn Levy
2,2010,Knight and Day,Naomi,James Mangold
3,2011,Fast Five,Gisele Yashar,Justin Lin
4,2013,Fast & Furious 6,Gisele Yashar,Justin Lin
5,2014,Kicking Out Shoshana,Mirit Ben Harush,Shay Kanot
6,2015,Furious 7,Gisele Yashar,James Wan
7,2016,Batman v Superman: Dawn of Justice,Diana Prince / Wonder Woman,Zack Snyder
8,2016,Criminal,Jill Pope,Ariel Vromen
9,2016,Keeping Up with the Joneses,Natalie Jones,Greg Mottola


In [8]:
#start answer to question 2
co_actors = dict()


In [9]:
# get all the cast from all the movies
def get_cast(film_page_soup):
    cast = film_page_soup.find('span', {'id': re.compile("Cast")})
    h2 = cast.parent
    ul = h2.findNext(["ul"])
    return ul.findAll(["li"])

# get all the players from the as_themselves title
def get_as_themselves(film_page_soup):
    as_themselves = film_page_soup.find('span', {'id': re.compile("As_themselves")})
    h3 = as_themselves.parent
    ul = h3.findNext(["ul"])
    return ul.findAll(["li"])

# actors_li hold all the actors li from all the movies casts
actors_li=[]
for movie_link in movies_links:
    film_page = urlopen("https://en.wikipedia.org"+movie_link.get("href"))
    film_page_soup = BeautifulSoup(film_page)
    actors_li=actors_li+get_cast(film_page_soup)
    try:
        actors_li=actors_li+get_as_themselves(movie_link)
    except:
        continue
    
# print("num of actors= "+str(len(actors_li)))
# print(actors_li)

In [10]:
# get the link and the name from the actor li 
def get_actor_link_and_name(actor_li):
    actor_a=actor_li.find('a')
    if actor_a:
        actor_link=actor_a.get("href")
        actor_name=actor_a.get("title")
        actor_link="https://en.wikipedia.org"+actor_link
        return actor_name, actor_link
    else:
        return 

# actors_links hold all the actors that has link from all the actors_li
dict_names_links={}

for actor_li in actors_li:
    link_and_name=get_actor_link_and_name(actor_li)
    if link_and_name is None:
        link_and_name=[]
        actor_li_str=str(actor_li)
        actor_li_str=actor_li_str.split('as')[0]
        actor_li_str=actor_li_str[4:]
        link_and_name.append(actor_li_str)
        link_and_name.append(None)
        if link_and_name[0] in co_actors.keys():
               co_actors[link_and_name[0]] += 1
        else:
            co_actors[link_and_name[0]] = 1
    if not re.match("Gal Gadot", link_and_name[0]):
        dict_names_links.update({link_and_name[0]: link_and_name[1]})
        if link_and_name[0] in co_actors.keys():
               co_actors[link_and_name[0]] += 1
        else:
            co_actors[link_and_name[0]] = 1
    
    


In [ ]:
#get the details of the actor: year of birth, country of birth, num of rewards.
def get_year_of_birth(actor_page_soup):
    try:
        year_of_birth = actor_page_soup.find('span', {'class': 'bday'}).getText()[:4]
    except:
        year_of_birth = 'NA'
    return year_of_birth

def get_country_of_birth(actor_page_soup):
    try:
        place_of_birth = actor_page_soup.find('div', {'class': 'birthplace'})
        place_of_birth = place_of_birth.getText()
#         print(place_of_birth)
        idx = place_of_birth.find(',')
        country_of_birth = None
        if idx!=-1:
            country_of_birth = place_of_birth[(idx + 2):]
            idx = country_of_birth.find(',')
            while idx!=-1:
                country_of_birth = country_of_birth[(idx + 2):] 
                idx = country_of_birth.find(',')
        else:
            country_of_birth=place_of_birth
    except:        
        country_of_birth = 'NA'
    return country_of_birth
    
def get_number_of_awards(actor_page_soup):
    num_of_awards=0

    Awards_and_nominations = actor_page_soup.find('span', {'id': re.compile("Awards_and_nominations")})
    if Awards_and_nominations is None:
        return num_of_awards
    h2 = Awards_and_nominations.parent
    table = h2.findNext('table', class_='wikitable sortable')
    if table is None:
        table = h2.findNext('table', class_='wikitable')
    if table is None:
        return num_of_awards
    for row in table.findAll("tr"):
        try:
            cell=row.findAll('td')
            isWon= cell[-1].find(text=True)
            if (str(isWon[0:3]) == 'Won'):
                num_of_awards+=1 
        except:
            continue
        
    return num_of_awards
    
    
    
def get_actor_details(actor_name,actor_link):
    #open the actor link
    try:
        actor_page = urlopen(actor_link)
    except:
#         print("cannot open link "+actor_name)
        return [actor_name, "NA", "NA",0]
    actor_page_soup = BeautifulSoup(actor_page, "lxml")
    
    #get the year of birth
    year_of_birth= get_year_of_birth(actor_page_soup)
        
    #get the country of birth
    country_of_birth=get_country_of_birth(actor_page_soup)
    
    #get the number of awards the actor won
    num_of_awards=get_number_of_awards(actor_page_soup)
        
#     print(actor_name+" "+ year_of_birth+" "+ country_of_birth+" "+str(num_of_awards))
    return [actor_name, year_of_birth, country_of_birth,num_of_awards]


#get the details for all the actors
actors_details=[]
for key in dict_names_links:
    new_details=get_actor_details(key,dict_names_links[key])
    if new_details!= None:
        actors_details.append(new_details)
     



In [ ]:
df_actors_data = pd.DataFrame(actors_details, columns = ['ActorName', 'YearOfBirth','CountryOfBirth','NumOfAwards'])
pd.set_option('display.max_rows', len(df_actors_data))
# df_actors_data=df_actors_data.sort_values('ActorName')
df_actors_data

In [ ]:
#answer to question 3

import numpy as np

df_co_actors=pd.DataFrame(co_actors.items(), columns=['ActorName', 'NumOfMovies'])

count_joint_movies = Counter(co_actors.values())
num = sorted(count_joint_movies.items())
objects = num
properties = [x[1] for x in num]
yint = np.arange(len(objects))
locations = [idx for idx, val in num]
plt.bar(yint, properties, align='center', alpha=0.5)
plt.xticks(yint, locations)
plt.title("Distribution of joint movies")
plt.xlabel("Joint movies")
plt.ylabel("Number of actors")
plt.show()
df_co_actors